# Simple trading system

First I researched possible simple trading strategies for a single stock and came to the conclusion that the best strategies would be
 - Mean reversion
 - Momentum trading
 
from these 2 I decided to implement **mean reversion**
 
 
 
 

# Mean reversion

Mean-reversion strategies work on the assumption that the price of an asset is prone to random fluctuation around an underlying stable trend. Therefore, values deviating far from the trend or observed mean will tend to reverse direction and revert to the mean. If the value is unusually high, we expect it to go back down and go up if it is unusually low.

# Single-stock mean reversion

Mean reversion in the context of a stock price implies that periods of the price being far below the mean are followed by periods of the price going up, and vice versa. We can take advantage of this by buying the stock to go long when the price is lower than expected, and selling to go short when the price is higher than expected.

In [1]:
pip install plotly

In [2]:
pip install yfinance

     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 6.4 MB 13.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import yfinance as yf

In [4]:
TICKER_SYMBOL = 'SPY'
START = '2000-01-01'
END = '2017-12-31'

TRANSACTION_COST = 0.0025

In [5]:
spy_df = yf.download(TICKER_SYMBOL,
                     start=START,
                     end=END,
                     progress=False)

# check if data has been loaded correctly
print(spy_df.shape)
print(spy_df.head())

(4528, 6)
                 Open       High         Low     Close  Adj Close    Volume
Date                                                                       
2000-01-03  148.25000  148.25000  143.875000  145.4375  96.855026   8164300
2000-01-04  143.53125  144.06250  139.640625  139.7500  93.067413   8089800
2000-01-05  139.93750  141.53125  137.250000  140.0000  93.233917  12177900
2000-01-06  139.62500  141.50000  137.750000  137.7500  91.735519   6227200
2000-01-07  140.31250  145.75000  140.062500  145.7500  97.063171   8066500


In [6]:
# since we buy at close we are intersted in Close prices
close = spy_df["Close"]
date = spy_df.index

# calculate cumulative avg
spy_df["Cum_avg"] = close.expanding().mean()

In [8]:
# plot the data and cum avg
fig = px.line(x=date, y=close, labels={"x": "date", "y": "price(USD)"})
fig.add_scatter(x=date, y=spy_df.Cum_avg, name="Cumulative avarage")
fig.show()

In order to trade using this strategy, we need to quantify what it means for the price to be higher or lower than expected. It's useful to compute the z-score of the price on each day, which tells us how many standard deviations away from the mean a value is:
$$ z = \frac{x - \mu}{\sigma} $$

where $x$ is the value, $\mu$ is the mean of the data set, and $\sigma$ is its standard deviation. So a price with a z-score $> 1$ is more than one standard deviation above the mean, and we will sell short when this happens. If the price on a day has a z-score $< 1$, we will buy long. If the price is within half a standard deviation of the mean, we will clear all positions.


In [9]:
# calculating z-scores
spy_df["Z_score"] = (spy_df.Close - spy_df.Cum_avg) / np.std(spy_df.Close)

In [18]:
# storing the buy/sell prices and the date of purchase, so we can plot later
sell_dict = {}
buy_dict = {}

# starting money in USD
money = 100000
stocks_owned = 0
count_stocks_to_buy = 500
overall_net_worth = money

# list containing our net worth for each day
daily_net_worth_list = []

for index, _ in enumerate(close):
    # Sell short if the z-score is > 1

    if spy_df["Z_score"][index] > 1 and stocks_owned > 0:
        money_exchanged = spy_df["Close"][index] * count_stocks_to_buy

        money += money_exchanged - (money_exchanged * TRANSACTION_COST)
        stocks_owned -= 1 * count_stocks_to_buy
        sell_dict[spy_df.index[index]] = spy_df["Close"][index]

    # Buy long if the z-score is < -1
    elif spy_df["Z_score"][index] < -1:
        money_exchanged = spy_df["Close"][index] * count_stocks_to_buy

        if money - spy_df["Close"][index] * count_stocks_to_buy > 0:
            money -= money_exchanged - (money_exchanged * TRANSACTION_COST)
            stocks_owned += 1 * count_stocks_to_buy

            buy_dict[spy_df.index[index]] = spy_df["Close"][index]

    # Clear positions if the z-score between -.5 and .5
    elif abs(spy_df["Z_score"][index]) < 0.5 and stocks_owned > 0:
        money_exchanged = stocks_owned * spy_df["Close"][index]

        money += money_exchanged - (money_exchanged * TRANSACTION_COST)
        stocks_owned = 0
        sell_dict[spy_df.index[index]] = spy_df["Close"][index]

    overall_net_worth = money + (stocks_owned * spy_df["Close"][index])
    daily_net_worth_list.append(overall_net_worth)

In [19]:
price_fig = [
    go.Candlestick(
        x=spy_df.index,
        open=spy_df["Open"],
        high=spy_df["High"],
        low=spy_df["Low"],
        close=spy_df["Close"],
        name=TICKER_SYMBOL,
    )
]

signals_fig = [
    go.Scatter(
        # unpacking to tuple, as scatter does not support dicts
        x=tuple(buy_dict.keys()),
        y=tuple(buy_dict.values()),
        mode="markers",
        hovertext="buy",
        name="buy",
        marker_size=9,
        marker_color="green",
    ),
    go.Scatter(
        x=tuple(sell_dict.keys()),
        y=tuple(sell_dict.values()),
        mode="markers",
        hovertext="sell",
        name="sell",
        marker_size=9,
        marker_color="red",
    ),
]

fig = go.Figure(data=price_fig + signals_fig)
fig.update_yaxes(fixedrange=False)  # unlock vertical scrolling
fig.update_layout(title="SPY stock prices with buy and sell points")
fig.show()

In [20]:
# performance chart
fig = px.line(
    daily_net_worth_list,
    x=spy_df.index,
    y=daily_net_worth_list,
    labels={"x": "date", "y": "value(USD)"},
    title="Overall Value",
)
fig.show()

# Conclusion

My approach made a net profit of 47K USD, however it only made 6 trades in a span of 18 years, which is not very much. Using this method on multiple stocks would have made a positive difference.